# Healthcare No Show Datasets Preparation

Prepare datasets for model training. By default, will prepare 3 for cross validation.

In [22]:
import sys
sys.path.append("../..")  # add src to path to import custom modules

import os
import dotenv
dotenv.load_dotenv()

import sqlalchemy
import pandas as pd
pd.set_option('display.expand_frame_repr', False)

import torch
from torch.utils.data import random_split
from src.dataset import CustomDataset

## Load data from SQL to pandas

In [2]:
engine = sqlalchemy.create_engine(
    f"mysql+mysqlconnector://{os.getenv('USER')}:{os.getenv('PASSWORD')}@{os.getenv('HOST')}:{os.getenv('PORT')}/{os.getenv('DATABASE')}"
)

In [3]:
meta_data = sqlalchemy.MetaData()
meta_data.reflect(bind=engine)
HEALTHCARE = meta_data.tables['healthcare']
query = sqlalchemy.select(
    HEALTHCARE.c.gender,
    HEALTHCARE.c.scheduled_day,
    HEALTHCARE.c.appointment_day,
    HEALTHCARE.c.age,
    HEALTHCARE.c.neighbourhood,
    HEALTHCARE.c.scholarship,
    HEALTHCARE.c.hypertension,
    HEALTHCARE.c.diabetes,
    HEALTHCARE.c.alcoholism,
    HEALTHCARE.c.handicap,
    HEALTHCARE.c.sms_received,
    HEALTHCARE.c.no_show
)

In [4]:
df = pd.read_sql_query(query, engine)
print(df)

       gender       scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show
0           F 2015-11-10 07:13:56      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0
1           M 2015-12-03 08:17:28      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1
2           F 2015-12-07 10:40:59      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1
3           F 2015-12-07 10:42:42      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0
4           F 2015-12-07 10:43:01      2016-06-03  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0
...       ...                 ...             ...   ...             ...     

## Data cleaning

Data representations for machine learning:
1. patient_id
    - no need
2. appointment_id
    - no need
3. gender
    - encode as a binary with 0 as male and 1 as female
4. scheduled_day
    - calculate lag days with appointment_day, then group them into ["same day", "7 days", "14 days", "30 days", "60 days", "90 days", ">90 days"], and use one-hot encoding
5. appointment_day
    - see scheduled_day
    - Also, convert to day of week
6. age
    - remove negatives
    - group into ["Infant", "Toddler", "Child", "Teen", "Adult", "Middle", "Senior"] and use one-hot encoding
7. neighbourhood
    - one-hot encoding
8. scholarship
    - binary
9. hypertension
    - binary
10. diabetes
    - binary
11. alcoholism
    - binary
12. handicap
    - one-hot encoding
13. sms_received
    - binary
14. no_show (this is the output)
    - binary

In [5]:
df["gender"] = df["gender"].apply(lambda x: 1 if x == "F" else 0)
print(df)

        gender       scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show
0            1 2015-11-10 07:13:56      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0
1            0 2015-12-03 08:17:28      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1
2            1 2015-12-07 10:40:59      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1
3            1 2015-12-07 10:42:42      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0
4            1 2015-12-07 10:43:01      2016-06-03  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0
...        ...                 ...             ...   ...             .

In [6]:
df["scheduled_day"] = df["scheduled_day"].dt.normalize()
df["appointment_day"] = df["appointment_day"].dt.normalize()
df["lag_days"] = (df["appointment_day"] - df["scheduled_day"]).dt.days
print(df)

        gender scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  lag_days
0            1    2015-11-10      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0       176
1            0    2015-12-03      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1       151
2            1    2015-12-07      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1       179
3            1    2015-12-07      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0       179
4            1    2015-12-07      2016-06-03  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0       179
...        ...           ...             ...  

In [7]:
lag_days_ranges = {
    "same day": 0.99,
    "7 days": 7,
    "14 days": 14,
    "30 days": 30,
    "60 days": 60,
    "90 days": 90,
    ">90 days": 200
}
labels, bins = zip(*lag_days_ranges.items())
df["lag_days_bins"] = pd.cut(df["lag_days"], bins=(0,)+bins, labels=labels, include_lowest=True)
print(df)

        gender scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  lag_days lag_days_bins
0            1    2015-11-10      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0       176      >90 days
1            0    2015-12-03      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1       151      >90 days
2            1    2015-12-07      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1       179      >90 days
3            1    2015-12-07      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0       179      >90 days
4            1    2015-12-07      2016-06-03  80.0   SÃO CRISTÓVÃO            0             1         1           0         0          

In [8]:
lag_days_keys = list(lag_days_ranges.keys())
df["lag_days_bins"] = df["lag_days_bins"].apply(lambda x: lag_days_keys.index(x))
print(df)

        gender scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  lag_days lag_days_bins
0            1    2015-11-10      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0       176             6
1            0    2015-12-03      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1       151             6
2            1    2015-12-07      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1       179             6
3            1    2015-12-07      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0       179             6
4            1    2015-12-07      2016-06-03  80.0   SÃO CRISTÓVÃO            0             1         1           0         0          

In [9]:
df["dayofweek"] = df["appointment_day"].dt.day_name()
print(df)

        gender scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  lag_days lag_days_bins  dayofweek
0            1    2015-11-10      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0       176             6  Wednesday
1            0    2015-12-03      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1       151             6     Monday
2            1    2015-12-07      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1       179             6     Friday
3            1    2015-12-07      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0       179             6     Friday
4            1    2015-12-07      2016-06-03  80.0   SÃO CRISTÓVÃO            0 

In [10]:
dayofweek_all = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5}
df["dayofweek"] = df["dayofweek"].apply(lambda x: dayofweek_all[x])
print(df)

        gender scheduled_day appointment_day   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  lag_days lag_days_bins  dayofweek
0            1    2015-11-10      2016-05-04  51.0     RESISTÊNCIA            0             0         0           0         0             1        0       176             6          2
1            0    2015-12-03      2016-05-02  34.0      VILA RUBIM            0             1         0           0         0             1        1       151             6          0
2            1    2015-12-07      2016-06-03  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1       179             6          4
3            1    2015-12-07      2016-06-03  48.0         MARUÍPE            0             1         1           0         0             1        0       179             6          4
4            1    2015-12-07      2016-06-03  80.0   SÃO CRISTÓVÃO            0 

In [11]:
df.drop(columns=["scheduled_day", "appointment_day", "lag_days"], inplace=True)
print(df)

        gender   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek
0            1  51.0     RESISTÊNCIA            0             0         0           0         0             1        0             6          2
1            0  34.0      VILA RUBIM            0             1         0           0         0             1        1             6          0
2            1  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1             6          4
3            1  48.0         MARUÍPE            0             1         1           0         0             1        0             6          4
4            1  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0             6          4
...        ...   ...             ...          ...           ...       ...         ...       ...           ...      ...           ...    

In [12]:
df = df[df["age"] >= 0]
print(df)

        gender   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek
0            1  51.0     RESISTÊNCIA            0             0         0           0         0             1        0             6          2
1            0  34.0      VILA RUBIM            0             1         0           0         0             1        1             6          0
2            1  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1             6          4
3            1  48.0         MARUÍPE            0             1         1           0         0             1        0             6          4
4            1  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0             6          4
...        ...   ...             ...          ...           ...       ...         ...       ...           ...      ...           ...    

In [13]:
age_ranges = {
    "Infant": 1,
    "Toddler": 4,
    "Child": 12,
    "Teen": 19,
    "Adult": 39,
    "Middle": 59,
    "Senior": 120
}
labels, bins = zip(*age_ranges.items())
df["age_group"] = pd.cut(df["age"], bins=(0,)+bins, labels=labels, include_lowest=True)
print(df)

        gender   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek age_group
0            1  51.0     RESISTÊNCIA            0             0         0           0         0             1        0             6          2    Middle
1            0  34.0      VILA RUBIM            0             1         0           0         0             1        1             6          0     Adult
2            1  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1             6          4     Adult
3            1  48.0         MARUÍPE            0             1         1           0         0             1        0             6          4    Middle
4            1  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0             6          4    Senior
...        ...   ...             ...          ...           ...       ...   

In [14]:
age_keys = list(age_ranges.keys())
df["age_group"] = df["age_group"].apply(lambda x: age_keys.index(x))
print(df)

        gender   age   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek age_group
0            1  51.0     RESISTÊNCIA            0             0         0           0         0             1        0             6          2         5
1            0  34.0      VILA RUBIM            0             1         0           0         0             1        1             6          0         4
2            1  27.0   SÃO CRISTÓVÃO            1             0         0           0         0             1        1             6          4         4
3            1  48.0         MARUÍPE            0             1         1           0         0             1        0             6          4         5
4            1  80.0   SÃO CRISTÓVÃO            0             1         1           0         0             1        0             6          4         6
...        ...   ...             ...          ...           ...       ...   

In [15]:
df.drop(columns=["age"], inplace=True)
print(df)

        gender   neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek age_group
0            1     RESISTÊNCIA            0             0         0           0         0             1        0             6          2         5
1            0      VILA RUBIM            0             1         0           0         0             1        1             6          0         4
2            1   SÃO CRISTÓVÃO            1             0         0           0         0             1        1             6          4         4
3            1         MARUÍPE            0             1         1           0         0             1        0             6          4         5
4            1   SÃO CRISTÓVÃO            0             1         1           0         0             1        0             6          4         6
...        ...             ...          ...           ...       ...         ...       ...           ...      ...

In [16]:
cities = sorted(df["neighbourhood"].unique())
df["neighbourhood"] = df["neighbourhood"].apply(lambda x: cities.index(x))
print(df)

        gender  neighbourhood  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show lag_days_bins  dayofweek age_group
0            1             59            0             0         0           0         0             1        0             6          2         5
1            0             80            0             1         0           0         0             1        1             6          0         4
2            1             75            1             0         0           0         0             1        1             6          4         4
3            1             44            0             1         1           0         0             1        0             6          4         5
4            1             75            0             1         1           0         0             1        0             6          4         6
...        ...            ...          ...           ...       ...         ...       ...           ...      ...       

In [17]:
df = pd.get_dummies(df, columns=["neighbourhood", "lag_days_bins", "dayofweek", "age_group"], dtype=int)
print(df)

        gender  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  no_show  neighbourhood_0  neighbourhood_1  ...  dayofweek_3  dayofweek_4  dayofweek_5  age_group_0  age_group_1  age_group_2  age_group_3  age_group_4  age_group_5  age_group_6
0            1            0             0         0           0         0             1        0                0                0  ...            0            0            0            0            0            0            0            0            1            0
1            0            0             1         0           0         0             1        1                0                0  ...            0            0            0            0            0            0            0            1            0            0
2            1            1             0         0           0         0             1        1                0                0  ...            0            1            0            0            0  

In [18]:
df = df[[c for c in df if c not in ["no_show"]] + ["no_show"]]
print(df)

        gender  scholarship  hypertension  diabetes  alcoholism  handicap  sms_received  neighbourhood_0  neighbourhood_1  neighbourhood_2  ...  dayofweek_4  dayofweek_5  age_group_0  age_group_1  age_group_2  age_group_3  age_group_4  age_group_5  age_group_6  no_show
0            1            0             0         0           0         0             1                0                0                0  ...            0            0            0            0            0            0            0            1            0        0
1            0            0             1         0           0         0             1                0                0                0  ...            0            0            0            0            0            0            1            0            0        1
2            1            1             0         0           0         0             1                0                0                0  ...            1            0            0        

## Create and Store Dataset

In [ ]:
def create_and_store_dataset(idx):
    full_dataset = CustomDataset(df)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    torch.save(
        {
            "train_dataset": train_dataset,
            "val_dataset": val_dataset
        },
        f"../../data/healthcare_no_show/healthcare_datasets_{idx}.pt"
    )

In [25]:
n_cross_val = 3
for i in range(n_cross_val):
    create_and_store_dataset(i)
    print(f"Dataset {i+1}/{n_cross_val} created and stored.")

Dataset 1/3 created and stored.
Dataset 2/3 created and stored.
Dataset 3/3 created and stored.
